In [44]:
import json
from fuzzywuzzy import fuzz
import re
import string
import torch
import os
import random
import copy

In [8]:
def normalize_answer(s):
    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def remove_citations(sent):
    return re.sub(r"\[\d+", "", re.sub(r" \[\d+", "", sent)).replace(" |", "").replace("]", "")

In [15]:
files = ['/home/shanghong/work/hallucination_app/best_model/asqa-checkpoint-90-gtr-shot2-ndoc5-42-temp0.1-no_demo.json',
         '/home/shanghong/work/hallucination_app/best_model/eli5-checkpoint-90-bm25-shot2-ndoc5-42-temp0.1-no_demo.json',
         '/home/shanghong/work/hallucination_app/best_model/qampari-checkpoint-90-gtr-shot2-ndoc5-42-temp0.1-no_demo.json']


# '/home/shanghong/work/hallucination_app/best_baseline/asqa-Pengfei-shot2-ndoc5-42-temp0.5.json',
#          '/home/shanghong/work/hallucination_app/best_baseline/eli5-Pengfei-shot2-ndoc5-42-temp0.5.json',
#          '/home/shanghong/work/hallucination_app/best_baseline/qampari-Pengfei-shot2-ndoc5-42-temp0.5.json',
         

In [21]:
REJECTION_FUZZ_THRESHOLD=85
REJECTION_FLAG="I apologize, but I couldn't find an answer"

llama3_human = []
for fn in files:
    with open(fn, 'r') as file:
        data = json.load(file)['data']
    # data = json.load(file)
    basename = os.path.basename(fn)
    print(f'Processing {basename}')

    rejection = []
    answered = []
    l = 0
    for item in data:
        item['label'] = l
        l+=1
        is_rejection = fuzz.partial_ratio(normalize_answer(REJECTION_FLAG), normalize_answer(item['output'])) > REJECTION_FUZZ_THRESHOLD
        if is_rejection:
            item['origin'] = basename
            item['reject'] = True
            rejection.append(item)
        else:
            item['origin'] = basename
            item['reject'] = False
            answered.append(item)
    
    print(f'Num rejection data: {len(rejection)}, answered data: {len(answered)}')
    final_rejection = random.sample(rejection, 5)
    final_answered = random.sample(answered, 5)
    
    print({len(final_rejection=)})
    print(len(final_answered))
    llama3_human.extend(final_rejection)
    llama3_human.extend(final_answered)

# random.shuffle(llama3_human)
print(f'Num of samples for human eval: {len(llama3_human)}')

Processing asqa-checkpoint-90-gtr-shot2-ndoc5-42-temp0.1-no_demo.json
Num rejection data: 413, answered data: 535
5
5
Processing eli5-checkpoint-90-bm25-shot2-ndoc5-42-temp0.1-no_demo.json
Num rejection data: 845, answered data: 155
5
5
Processing qampari-checkpoint-90-gtr-shot2-ndoc5-42-temp0.1-no_demo.json
Num rejection data: 776, answered data: 224
5
5
Num of samples for human eval: 30


In [29]:
with open('llama3_human_eval_data.json', 'w', encoding='utf-8') as f:
    json.dump(llama3_human, f, indent=4)

In [23]:
files = ['/home/shanghong/work/hallucination_app/best_baseline/asqa-Pengfei-shot2-ndoc5-42-temp0.5.json',
         '/home/shanghong/work/hallucination_app/best_baseline/eli5-Pengfei-shot2-ndoc5-42-temp0.5.json',
         '/home/shanghong/work/hallucination_app/best_baseline/qampari-Pengfei-shot2-ndoc5-42-temp0.5.json']

In [39]:
baseline_human = []
i = 0
fp = 0
for llama3_item in llama3_human:
    print(i)
    if (i == 0 or (i % 10 == 0)):
        fn = files[fp]
        with open(fn, 'r') as file:
            data = json.load(file)['data']
        fp += 1
        basename = os.path.basename(fn)
        print(f'Processing {basename}')


    label = llama3_item['label']
    reject = llama3_item['reject']
    item = data[label]
    item['label'] = label
    item['reject'] = reject
    item['origin'] = basename
    baseline_human.append(item)
    i+=1
    
        

0
Processing asqa-Pengfei-shot2-ndoc5-42-temp0.5.json
1
2
3
4
5
6
7
8
9
10
Processing eli5-Pengfei-shot2-ndoc5-42-temp0.5.json
11
12
13
14
15
16
17
18
19
20
Processing qampari-Pengfei-shot2-ndoc5-42-temp0.5.json
21
22
23
24
25
26
27
28
29


In [40]:
with open('gpt35_human_eval_data.json', 'w', encoding='utf-8') as f:
    json.dump(baseline_human, f, indent=4)

In [45]:
output = []
for item1, item2 in zip(llama3_human, baseline_human):
    assert item1['label'] == item2['label']
    assert item1['question'] == item2['question'], print(f'{item1['question']}, {item2['question']}')
    item = copy.deepcopy(item1)
    del item['output']
    item['llama3_answer'] = item1['output']
    item['gpt35_answer'] = item2['output']
    output.append(item)

In [46]:
with open('output.json', 'w', encoding='utf-8') as f:
    json.dump(output, f, indent=4)

In [22]:
print(f'Question: {human_eval_data[0]["question"]}')
print("=======")
i = 1
for doc in human_eval_data[0]['docs']:
    print(f'[Doc {i}]')
    print(f'Title: {doc["title"]}')
    print(f'Text: {doc["text"]}')
    print('----')
    i+=1
print("=======")
print(f'Output: {human_eval_data[0]["output"]}')

Question: What are the positive aspects of the American National Debt, if any, and what would be the benefits of paying off the debt?
[Doc 1]
Title: national debt | a whisper in the wind
Text: interest. We could establish a way for people to contribute to paying off the principle of the national debt by their donations. This first year, all of those people who make over 250,000 who voted for Obama could voluntarily contribute the difference between what they pay for taxes under Bush and what they will be paying under Obama. That would be a nice start, and it really wouldn’t cost them any more than what they have offered to pay the government anyway. Plus, there are many average Americans who recognize the need to pay down our debt, who might just
----
[Doc 2]
Title: What if we’d been on the gold standard? | Econbrowser
Text: increase the supply of currency only by paying off some of that debt. If they did so by buying gold instead, the level of debt would not be reduced. This is someth

In [13]:
docs

'interest. We could establish a way for people to contribute to paying off the principle of the national debt by their donations. This first year, all of those people who make over 250,000 who voted for Obama could voluntarily contribute the difference between what they pay for taxes under Bush and what they will be paying under Obama. That would be a nice start, and it really wouldn’t cost them any more than what they have offered to pay the government anyway. Plus, there are many average Americans who recognize the need to pay down our debt, who might justincrease the supply of currency only by paying off some of that debt. If they did so by buying gold instead, the level of debt would not be reduced. This is something of a zero sum game here; any benefit to holders of gold in the form of higher prices would come at the direct expense of taxpayers in the form of higher claims on their future earnings in the form of national debt.” Perhaps I am missing something but when exactly did t

# Form creation

In [66]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Define the scope and your credentials file
SCOPES = [
    'https://www.googleapis.com/auth/forms.body',
    'https://www.googleapis.com/auth/forms.responses.readonly'
]
SERVICE_ACCOUNT_FILE = '/home/maojia/work/Cite-PM/data/evaluation/human_eval/hallucination-432518-852da74548d1.json'

# Authenticate and create the service
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('forms', 'v1', credentials=creds)

# Form ID of the Google Form you want to retrieve responses from
form_id = "1S2CVoU2pYJq-wl9zkFQGq84Sj8YNfalZuRHoxo26ezc"

# Retrieve responses from the form
result = service.forms().responses().list(formId=form_id).execute()
print(result)


{'responses': [{'responseId': 'ACYDBNjbWjyk1rT9f_wpIvBP-GqoxTtqs071aQj1NO87RU5sWGpgorfuWHvleuBI5YHVhBc', 'createTime': '2024-08-14T19:08:22.191Z', 'lastSubmittedTime': '2024-08-14T19:08:22.191704Z', 'answers': {'4d818704': {'questionId': '4d818704', 'textAnswers': {'answers': [{'value': 'Mustard'}]}}}}, {'responseId': 'ACYDBNhTSDbyEOwiEXkXUx4d-ZSTxv6Nc2UPomYeAzeYMMngiKuoNT0ZqG1p_MOQp73NY6c', 'createTime': '2024-08-14T19:06:42.958Z', 'lastSubmittedTime': '2024-08-14T19:06:42.958406Z', 'answers': {'4d818704': {'questionId': '4d818704', 'textAnswers': {'answers': [{'value': 'Ketchup'}]}}}}, {'responseId': 'ACYDBNi_Xvi2SaveQNRE9uaQIO53RDZoa-dYKLXhe6a7dORSoTXNdBqxHvu63-QkBksq1EY', 'createTime': '2024-08-14T19:08:26.171Z', 'lastSubmittedTime': '2024-08-14T19:08:26.171466Z', 'answers': {'4d818704': {'questionId': '4d818704', 'textAnswers': {'answers': [{'value': 'Ketchup'}]}}}}]}


In [7]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Define the scope and the credentials file
SCOPES = ['https://www.googleapis.com/auth/forms.body']
CLIENT_SECRETS_FILE = '/home/maojia/work/Cite-PM/data/evaluation/human_eval/client_secret_151971149418-q1q08rqkpr46dvpij67du7k949h591ki.apps.googleusercontent.com.json'

# Run the OAuth flow to get credentials
flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
creds = flow.run_local_server(port=8080)  # This will use http://localhost:8080/ as redirect_uri

# Build the service using the credentials
service = build('forms', 'v1', credentials=creds)

# Create a new form
form = {
    "info": {
        "title": "New Form",
        "documentTitle": "New Form"
    }
}

# Call the forms API to create the form
created_form = service.forms().create(body=form).execute()
print(f'Form created with ID: {created_form["formId"]}')


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=151971149418-q1q08rqkpr46dvpij67du7k949h591ki.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fforms.body&state=ZKK4wFiarDRQ2lUS7wxVwsS475XExU&access_type=offline
Form created with ID: 1CKDJKXRVJ_ye2A0Jf7_RYR8t_laLfLUtT56c9UDVc14


In [73]:
formId = "1meMZaZNyToYxZCVDhVCitdtna52qPOF18ZSm1GcxA1M"

# Call the forms API to create the form
created_form = service.forms().create(body=form).execute()
print(f'Form created with ID: {created_form["formId"]}')

# Add a checkbox question
checkbox_question = {
    "requests": [{
        "createItem": {
            "item": {
                "title": "What condiments would you like on your hot dog?",
                "questionItem": {
                    "question": {
                        "required": True,
                        "choiceQuestion": {
                            "type": "CHECKBOX",
                            "options": [
                                {"value": "Ketchup"},
                                {"value": "Mustard"},
                                {"value": "Relish"}
                            ],
                        }
                    }
                }
            },
            "location": {
                "index": 0
            }
        }
    }]
}

# Add the checkbox question to the form
service.forms().batchUpdate(formId=created_form["formId"], body=checkbox_question).execute()


print(f"Form created successfully: https://docs.google.com/forms/d/{formId}/edit")

Form created with ID: 1doBbnSdAykY3SRZOGb2969wUeAkoAfsguIrJQ1DvPqc
Form created successfully: https://docs.google.com/forms/d/1meMZaZNyToYxZCVDhVCitdtna52qPOF18ZSm1GcxA1M/edit


In [23]:
from googleapiclient.discovery import build

# Assuming 'service' is already authenticated and the form is created
form_id = '1doBbnSdAykY3SRZOGb2969wUeAkoAfsguIrJQ1DvPqc'  # Replace with your actual form ID

# Define the batch update request to add questions
batch_update_request = {
    "requests": [
        {
            "createItem": {
                "item": {
                    "title": "<br>What condiments would you <br> like on your hot dog?",
                    "questionItem": {
                        "question": {
                            "required": True,
                            "choiceQuestion": {
                                "type": "CHECKBOX",
                                "options": [
                                    {"value": "Ketchup"},
                                    {"value": "Mustard"},
                                    {"value": "Relish"}
                                ],
                            }
                        }
                    }
                },
                "location": {
                    "index": 0
                }
            }
        },
        {
            "createItem": {
                "item": {
                    "title": "Do you prefer cats or dogs?",
                    "questionItem": {
                        "question": {
                            "required": True,
                            "choiceQuestion": {
                                "type": "RADIO",
                                "options": [
                                    {"value": "Cats"},
                                    {"value": "Dogs"},
                                ],
                                "shuffle": False
                            }
                        }
                    }
                },
                "location": {
                    "index": 1
                }
            }
        },
        {
            "createItem": {
                "item": {
                    "title": "When were you born?",
                    "questionItem": {
                        "question": {
                            "required": True,
                            "dateQuestion": {}
                        }
                    }
                },
                "location": {
                    "index": 2
                }
            }
        }
    ]
}

# Execute the batch update request to add the questions
response = service.forms().batchUpdate(formId=form_id, body=batch_update_request).execute()

print("Questions added successfully:", response)


HttpError: <HttpError 400 when requesting https://forms.googleapis.com/v1/forms/1doBbnSdAykY3SRZOGb2969wUeAkoAfsguIrJQ1DvPqc:batchUpdate?alt=json returned "Displayed text cannot contain newlines". Details: "Displayed text cannot contain newlines">